In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys

sys.path.append('/content/gdrive/My Drive/Colab Notebooks/ECE_Project')

In [0]:
import CNN_Lib
import numpy as np 
import torch
from torch.autograd import Variable
import random
import pdb



In [0]:
dir = '/content/gdrive/My Drive/Colab Notebooks/ECE_Project'
X_test = np.load(dir + '/X_test.npy')
y_test = np.load(dir + '/y_test.npy')
person_train_valid = np.load(dir + '/person_train_valid.npy')
X_train_valid = np.load(dir + '/X_train_valid.npy')
y_train_valid = np.load(dir + '/y_train_valid.npy')
person_test = np.load(dir + '/person_test.npy')
print ('Training/Valid data shape: {}' .format(X_train_valid.shape))
print ('Test data shape: {}' .format(X_test.shape))
print ('Training/Valid target shape: {}' .format(y_train_valid.shape))
print ('Test target shape: {}' .format(y_test.shape))
print ('Person train/valid shape: {}' .format(person_train_valid.shape))
print ('Person test shape: {}' .format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
#transform data into torch-readable data types
#2115 number of trials from 9 people
#four possible classes of outputs
#Xtrain = Variable(torch.from_numpy(X_train_valid)).cuda()
#ytrain = Variable(torch.from_numpy(y_train_valid)).cuda()
#Xtest = Variable(torch.from_numpy(X_test)).cuda()
#ytest = Variable(torch.from_numpy(y_test)).cuda()

Xtrain = torch.from_numpy(X_train_valid)
ytrain = torch.from_numpy(y_train_valid)
Xtest = torch.from_numpy(X_test)
ytest = torch.from_numpy(y_test)



In [0]:
def map_to_class(input_labels):
  mask1 = (input_labels == 769)*0
  mask2 = (input_labels == 770)*1
  mask3 = (input_labels == 771)*2
  mask4 = (input_labels == 772)*3

  return (mask1 + mask2 + mask3 + mask4)

In [0]:
########## INITIALIZE CNN MODEL ########## 

Trials, InputDim, SeqDim = X_train_valid.shape
num_features = InputDim
Tests = y_test.shape


#-----CNN tweak here----#
num_filters = 64
batch_size = 650
num_classes = 4
L2 = 0.5
cnnfilter_stride = 2
cnnfilter_size = 3 
cnn_padding = 2
use_bias = False
pool_size = 4
pool_stride = 1
use_batchnorm = True
eps = 1e-4
momentum = 0.9
affine = True
dropout = 0.25

#-----OPTIM tweak here----#
learning_rate = 0.0001

cnn_model = CNN_Lib.EEG_CNN(InputDim, SeqDim, num_filters, cnnfilter_size, cnnfilter_stride, cnn_padding, use_bias, pool_size, pool_stride, num_classes, use_batchnorm, eps, momentum, affine, dropout)
cnn_model.to('cuda:0') #--- activate the GPU


EEG_CNN(
  (CNN): Conv1d(22, 64, kernel_size=(3,), stride=(2,), padding=(2,), bias=False)
  (RELU): ReLU()
  (BatchNorm): BatchNorm1d(64, eps=0.0001, momentum=0.9, affine=True, track_running_stats=True)
  (DropOut): Dropout(p=0.25, inplace=False)
  (MaxPool): MaxPool1d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
  (FC1): Linear(in_features=31872, out_features=498, bias=True)
  (FC2): Linear(in_features=498, out_features=4, bias=True)
)

In [0]:
########## CHOOSE LOSS  ###########
loss = torch.nn.CrossEntropyLoss()

######### CHOOSE OPTIMIZER ########
optim = torch.optim.Adam(cnn_model.parameters(), lr=learning_rate, weight_decay=L2)


In [0]:
######## TRAIN IT ########
epochs = 1
iterations = 150
iter_ = 0
for epoch in range(epochs):

  train_correct = 0
  train_total = 0
  
  ###DEBUG
  #pdb.set_trace()
  for i in range(iterations):
    #put the model in training mode
    cnn_model.train(True)

    #process the input data
    idx = np.random.randint(0, Trials, size=batch_size)
    #idx = range(0,batch_size)
    xtrain = Xtrain[idx].view(batch_size, InputDim, SeqDim)
    xtrain = xtrain.to('cuda:0').requires_grad_()

    #forward pass
    optim.zero_grad()
    outFC = cnn_model.forward(xtrain.float())

    pdb.set_trace()
    
    #map to classes
    classes = map_to_class(ytrain[idx].long())
    classes = classes.to('cuda:0')

    #backward pass (gradient calculation)
    probs = loss(outFC, classes)
    probs.backward()

    #update weights
    optim.step()

    #update iter counter
    iter_+=1

    #calculate training accuracy
    values, train_pred = torch.max(outFC, 1)
    train_correct += (1*(train_pred == classes)).sum()
    train_total += float(classes.size(0))

    if (iter_ % 10) == 0:
      cnn_model.eval()

      total = 0
      correct = 0
      for j in range(X_test.shape[0]):
        #process the input data
        idx = np.random.randint(0, Tests, size=batch_size)
        xtest = Xtest[idx].view(-1, InputDim, SeqDim)
        xtest = xtest.to('cuda:0')

        #forward prop
        predict = cnn_model(xtest.float())
        
        #predict
        values, predicted_classes = torch.max(predict, 1)
        expected_classes = map_to_class(ytest[idx])        
        correct += (1*(predicted_classes == expected_classes.to('cuda:0'))).sum()
        total +=float(predicted_classes.size(0))

      print(f"\ncorrect: {correct}, total: {total}")
      #print(f"prediction: {predicted_classes}\nexpected_classes: {expected_classes}")
      print(f"Iteration: {iter_}, Loss: {probs.item()}, Validation Accuracy: {100*(correct/total)}%, Training Accuracy: {100*(train_correct/train_total)}%")  

Size of CNN output torch.Size([650, 64, 501])
Size of ReLU output torch.Size([650, 64, 501])
Size of BatchNorm output torch.Size([650, 64, 501])
Size of DropOut output torch.Size([650, 64, 501])
Size of MaxPool output torch.Size([650, 64, 498])
Size of Flattened output torch.Size([650, 31872])
Size of FC1 output torch.Size([650, 498])
Size of FC2 output torch.Size([650, 4])
> <ipython-input-9-9f38d9089c69>(28)<module>()
-> classes = map_to_class(ytrain[idx].long())
(Pdb) q


BdbQuit: ignored